In [ ]:
import anndata
import numpy as np
import scvelo as scv
import sys
import torch
import os.path
sys.path.append('../')
import velovae as vv

In [ ]:
filename = '/scratch/blaauw_root/blaauw1/gyichen/output_braindev.h5ad'
adata = anndata.read_h5ad(filename)
#adata.obs['clusters'] = adata.obs['leiden'].to_numpy()
adata.layers

In [ ]:
Ngene = 1000
vv.preprocess(adata, Ngene)

In [ ]:
figure_path = '../figures/Braindev/Default'
model_path = '../checkpoints/Braindev/Default'
data_path = '/scratch/blaauw_root/blaauw1/gyichen'
adata.X.shape
#os.mkdir(figure_path)
#os.mkdir(model_path)

In [ ]:
def sampleGenes(adata, n):
    variable = adata.var['highly_variable'].to_numpy()
    total = adata.var['total_counts'].to_numpy()
    idx_sort = np.flip(np.argsort(total))
    genes = adata.var_names.to_numpy()
    #sort genes based total counts
    print(total[idx_sort][:10])
    variable = variable[idx_sort]
    genes = genes[idx_sort]
    
    genes = genes[variable]
    idx = np.random.choice(np.sum(variable), n, replace=False)
    return genes[idx]
gene_plot = sampleGenes(adata, 8)
gene_plot

In [ ]:
gene_plot = ['Auts2', 'Dync1i1', 'Gm3764', 'Mapt', 'Nfib', 'Rbfox1', 'Satb2', 'Slc6a13', 'Srrm4', 'Tcf4']#some marker genes
config_vae = {
    'num_epochs':800, 
    'test_epoch':50, 
    'save_epoch':50, 
    'learning_rate':2e-4, 
    'learning_rate_ode':2e-4, 
    'lambda':1e-8, 
    'reg_t':1.0, 
    'batch_size':128,
}

model = vv.VanillaVAE(adata, 20, hidden_size=(500, 250), tprior='tprior', device='gpu')

In [ ]:
model.train(adata, config=config_vae, gene_plot=gene_plot, figure_path=figure_path)

In [ ]:
model.saveModel(model_path)
model.saveAnnData(adata, 'vanilla', data_path, file_name='output_braindev.h5ad')